In [1]:
!nvidia-smi

Fri Sep 15 07:49:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/anilbhatt1/ERA1_S17_BERT_GPT_VIT.git

Cloning into 'ERA1_S17_BERT_GPT_VIT'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 54 (delta 14), reused 47 (delta 7), pack-reused 0
Receiving objects: 100% (54/54), 4.47 MiB | 4.69 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [3]:
import os
print(os.getcwd())
os.chdir('/content/ERA1_S17_BERT_GPT_VIT/assignment')
print(os.getcwd())

/content
/content/ERA1_S17_BERT_GPT_VIT/assignment


In [4]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.2 MB/s eta 0:00:00


In [5]:
from torch.utils.data import Dataset
import torch.nn.functional as F
from collections import Counter
from os.path import exists
import torch.optim as optim
import torch.nn as nn
import numpy as np
import random
import torch
import math
import re
import yaml
from transformer import *
from bert_data import *

In [6]:
# Load the YAML file
with open('/content/ERA1_S17_BERT_GPT_VIT/assignment/config.yaml', 'r') as yaml_file:
    cfg = yaml.load(yaml_file, Loader=yaml.FullLoader)
cfg['model'] = 'bert'
model_nm = cfg['model']
print(model_nm)
cfg

bert


{'model': 'bert',
 'bert': {'batch_size': 1024,
  'seq_len': 20,
  'embed_size': 128,
  'inner_ff_size': 512,
  'n_heads': 8,
  'n_code': 8,
  'n_vocab': 40000,
  'dropout': 0.1,
  'n_workers': 12,
  'optim_kwargs': {'lr': 0.0001,
   'weight_decay': 0.0001,
   'betas': [0.9, 0.999]},
  'sentence_path': '/content/ERA1_S17_BERT_GPT_VIT/course_docs/BERT/training.txt',
  'vocab_path': '/content/ERA1_S17_BERT_GPT_VIT/course_docs/BERT/vocab.txt',
  'data_kwargs': {'shuffle': True,
   'drop_last': False,
   'pin_memory': True,
   'batch_size': 1024},
  'train_iter': 2000,
  'print_iter': 10,
  'embed_save_limit': 3000}}

In [7]:
print('initializing from config data..')
batch_size = cfg[model_nm]['batch_size']
seq_len = cfg[model_nm]['seq_len']
embed_size = cfg[model_nm]['embed_size']
inner_ff_size = cfg[model_nm]['inner_ff_size']
n_heads = cfg[model_nm]['n_heads']
n_code = cfg[model_nm]['n_code']
n_vocab = cfg[model_nm]['n_vocab']
dropout = cfg[model_nm]['dropout']
# n_workers = cfg[model_nm]['n_workers']
sent_path = cfg[model_nm]['sentence_path']
vocab_path = cfg[model_nm]['vocab_path']

#optimizer
optim_kwargs = cfg[model_nm]['optim_kwargs']
data_kwargs = cfg[model_nm]['data_kwargs']
train_iter = cfg[model_nm]['train_iter']
print_iter = cfg[model_nm]['print_iter']
embed_save_limit = cfg[model_nm]['embed_save_limit']

initializing from config data..


In [8]:
# =============================================================================
# Input
# =============================================================================
#1) load text
print('loading text...')
sentences = open(sent_path).read().lower().split('\n')

#2) tokenize sentences (can be done during training, you can also use spacy udpipe)
print('tokenizing sentences...')
special_chars = ',?;.:/*!+-()[]{}"\'&'
sentences = [re.sub(f'[{re.escape(special_chars)}]', ' \g<0> ', s).split(' ') for s in sentences]
sentences = [[w for w in s if len(w)] for s in sentences]

#3) create vocab if not already created
print('creating/loading vocab...')
if not exists(vocab_path):
    print('vocab_path doesnt exist...')
    words = [w for s in sentences for w in s]
    vocab = Counter(words).most_common(n_vocab) #keep the N most frequent words
    vocab = [w[0] for w in vocab]
    open(vocab_path, 'w+').write('\n'.join(vocab))
else:
    print('vocab_path exists...')
    vocab = open(vocab_path).read().split('\n')

loading text...
tokenizing sentences...
creating/loading vocab...
vocab_path exists...


In [9]:
#4) create dataset
print('creating dataset...')
dataset = SentencesDataset(sentences, vocab, seq_len)
kwargs = cfg[model_nm]['data_kwargs']
data_loader = torch.utils.data.DataLoader(dataset, **kwargs)

creating dataset...


In [10]:
# =============================================================================
# Model
# =============================================================================
#init model
print('initializing model...')
model = bert_Transformer(n_code, n_heads, embed_size, inner_ff_size, len(dataset.vocab), seq_len, dropout)
model = model.cuda()

initializing model...


In [11]:
# =============================================================================
# Optimizer
# =============================================================================
print('initializing optimizer and loss...')
optimizer = optim.Adam(model.parameters(), **optim_kwargs)
loss_model = nn.CrossEntropyLoss(ignore_index=dataset.IGNORE_IDX)

initializing optimizer and loss...


In [12]:
# =============================================================================
# Train
# =============================================================================
print('training...')
model.train()
batch_iter = iter(data_loader)
for it in range(train_iter):

    #get batch
    batch, batch_iter = get_batch(data_loader, batch_iter)

    #infer
    masked_input = batch['input']
    masked_target = batch['target']

    masked_input = masked_input.cuda(non_blocking=True)
    masked_target = masked_target.cuda(non_blocking=True)
    output = model(masked_input)

    #compute the cross entropy loss
    output_v = output.view(-1,output.shape[-1])
    target_v = masked_target.view(-1,1).squeeze()
    loss = loss_model(output_v, target_v)

    #compute gradients
    loss.backward()

    #apply gradients
    optimizer.step()

    #print step
    if it % print_iter == 0:
        print('it:', it,
              ' | loss', np.round(loss.item(),2),
              ' | Δw:', round(model.embeddings.weight.grad.abs().sum().item(),3))

    #reset gradients
    optimizer.zero_grad()

training...
it: 0  | loss 10.32  | Δw: 1.144
it: 10  | loss 9.57  | Δw: 0.547
it: 20  | loss 9.34  | Δw: 0.343
it: 30  | loss 9.17  | Δw: 0.264
it: 40  | loss 9.0  | Δw: 0.231
it: 50  | loss 8.82  | Δw: 0.2
it: 60  | loss 8.72  | Δw: 0.181
it: 70  | loss 8.51  | Δw: 0.17
it: 80  | loss 8.36  | Δw: 0.169
it: 90  | loss 8.24  | Δw: 0.161
it: 100  | loss 8.11  | Δw: 0.148
it: 110  | loss 7.86  | Δw: 0.151
it: 120  | loss 7.77  | Δw: 0.136
it: 130  | loss 7.67  | Δw: 0.132
it: 140  | loss 7.53  | Δw: 0.132
it: 150  | loss 7.41  | Δw: 0.125
it: 160  | loss 7.34  | Δw: 0.121
it: 170  | loss 7.24  | Δw: 0.115
it: 180  | loss 7.1  | Δw: 0.116
it: 190  | loss 7.09  | Δw: 0.113
it: 200  | loss 6.92  | Δw: 0.11
it: 210  | loss 6.94  | Δw: 0.111
it: 220  | loss 6.87  | Δw: 0.108
it: 230  | loss 6.82  | Δw: 0.109
it: 240  | loss 6.79  | Δw: 0.107
it: 250  | loss 6.67  | Δw: 0.109
it: 260  | loss 6.61  | Δw: 0.112
it: 270  | loss 6.56  | Δw: 0.112
it: 280  | loss 6.57  | Δw: 0.117
it: 290  | loss 6.

In [13]:
# =============================================================================
# Results analysis
# =============================================================================
print('saving embeddings...')
N = embed_save_limit
np.savetxt('values.tsv', np.round(model.embeddings.weight.detach().cpu().numpy()[0:N], 2), delimiter='\t', fmt='%1.2f')
s = [dataset.rvocab[i] for i in range(N)]
open('names.tsv', 'w+').write('\n'.join(s) )
print('end')

saving embeddings...
end
